In [1]:
val path = s"/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
// note: use sbt assembly to create jar
interp.load.cp(os.Path(path))
import $ivy.`org.plotly-scala::plotly-almond:0.8.2`
import $ivy.`com.lihaoyi:ujson_2.13:1.3.8`
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`

import ujson.Value
import sys.process._

// interp.load.ivy("com.github.pathikrit" %% "better-files" % "3.9.1")
import $ivy.`com.github.pathikrit::better-files:3.9.1`
import better.files._
import $ivy.`com.lihaoyi::ujson:1.3.12`
import $ivy.`com.lihaoyi::requests:0.6.5`
import $ivy.`com.lihaoyi::scalatags:0.12.0`



path: String = "/home/bounder/target/scala-2.13/soot_hopper-assembly-0.1.jar"
import $ivy.$                                      

import $ivy.$                             
// import $ivy.`edu.colorado.plv.bounder:soot_hopper_2.13:0.1`


import ujson.Value

import sys.process._

// interp.load.ivy("com.github.pathikrit" %% "better-files" % "3.9.1")

import $ivy.$                                         

import better.files._

import $ivy.$                          

import $ivy.$                            

import $ivy.$                              



In [2]:
import scala.util.Random
import edu.colorado.plv.bounder.Driver
import edu.colorado.plv.bounder.RunConfig
import edu.colorado.plv.bounder.symbolicexecutor.state.PureConstraint
import upickle.default.read
import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}

import plotly._
import plotly.element._
import plotly.layout._
import plotly.Almond._
import upickle.default.{read, write}

import scala.util.Random

import edu.colorado.plv.bounder.Driver

import edu.colorado.plv.bounder.RunConfig

import edu.colorado.plv.bounder.symbolicexecutor.state.PureConstraint

import upickle.default.read

import scala.collection.parallel.CollectionConverters.{ImmutableSetIsParallelizable, IterableIsParallelizable}


import plotly._

import plotly.element._

import plotly.layout._

import plotly.Almond._

import upickle.default.{read, write}

In [3]:
import edu.colorado.plv.bounder.symbolicexecutor.state.DBOutputMode
import edu.colorado.plv.bounder.symbolicexecutor.state.{DBPathNode,IPathNode,PrettyPrinting}
import edu.colorado.plv.bounder.ir._
import edu.colorado.plv.bounder.symbolicexecutor.QueryFinished
import edu.colorado.plv.bounder.BounderUtil
def obsMessagesStr(nodes:List[DBPathNode]):List[DBPathNode] = {
nodes.flatMap{n =>
  n.qry.loc match {
    case c : CallinMethodReturn => Some(n)
    case c : CallinMethodInvoke => Some(n)
    case c : GroupedCallinMethodInvoke => Some(n)
    case c : GroupedCallinMethodReturn => Some(n)
    case c : CallbackMethodInvoke => Some(n)
    case c : CallbackMethodReturn => Some(n)
    case _ => None
  }}
}

// implicit val db = DBOutputMode("/Users/shawnmeier/Desktop/db_triage/connectbot1.db",true)
implicit val db = DBOutputMode("/home/notebooks/reachExpLarge/Connectbot/GeneratePubkeyActivity_5_run/paths.db")

import edu.colorado.plv.bounder.symbolicexecutor.state.DBOutputMode

import edu.colorado.plv.bounder.symbolicexecutor.state.{DBPathNode,IPathNode,PrettyPrinting}

import edu.colorado.plv.bounder.ir._

import edu.colorado.plv.bounder.symbolicexecutor.QueryFinished

import edu.colorado.plv.bounder.BounderUtil

defined function obsMessagesStr
db: DBOutputMode = DBOutputMode(
  "/home/notebooks/reachExpLarge/Connectbot/GeneratePubkeyActivity_5_run/paths.db"
)

In [4]:
// find the callbacks that show up the most often
object WorstCallbacks{
    val counts = db.getAllLiveNodes().filter{n => n.qry.loc.isInstanceOf[CallbackMethodReturn]}.groupBy{_.qry.loc}.map{l => (l._1.toString,l._2.size)}.toList.sortBy(_._2).reverse.take(20)
}
WorstCallbacks.counts

defined object WorstCallbacks
res3_1: List[(String, Int)] = List(
  (
    "[CB Ret] org.connectbot.GeneratePubkeyActivity$4 void onClick(android.view.View)",
    28
  ),
  (
    "[CB Ret] org.connectbot.GeneratePubkeyActivity$KeyGeneratorRunnable void run()",
    28
  ),
  (
    "[CB Ret] org.connectbot.util.EntropyView boolean onTouchEvent(android.view.MotionEvent)",
    28
  ),
  ("[CB Ret] org.connectbot.GeneratePubkeyActivity$5 void run()", 28),
  ("[CB Ret] org.connectbot.GeneratePubkeyActivity void onResume()", 27),
  ("[CB Ret] org.connectbot.GeneratePubkeyActivity void onPause()", 23),
  (
    "[CB Ret] org.connectbot.GeneratePubkeyActivity void onCreate(android.os.Bundle)",
    20
  ),
  (
    "[CB Ret] org.connectbot.util.EntropyView void <init>(android.content.Context,android.util.AttributeSet)",
    16
  )
)

In [24]:
// Get sampling of live queries and their histories

// File.usingTemporaryFile() {tempFile =>
//   val dbgCre:Set[IPathNode] = db.getAllNodes().filter{n => n.qry.state.toString.contains("setNegativeButton")}.take(4).toSet
//   // println(s"found ${dbgCre.size} nodes")
//   PrettyPrinting.printTraces(dbgCre, tempFile.toString, false)
//   // println(tempFile.contentAsString)
// } 
import scala.collection.mutable
import scala.util.Random
val traces = mutable.HashSet[String]()
// Random.shuffle(db.getAllNodes().filter{n => !n.qry.searchState.toString.contains("live")}).foreach{a => 
Random.shuffle(db.getNoSucc().filter{n => n.qry.searchState.toString.contains("live") && n.subsumed.isEmpty}).take(3).foreach{a => 
    println(PrettyPrinting.qryString(a.qry))
    // if (PrettyPrinting.filterForLocSeq(a, List())){
    //     File.usingTemporaryFile() {tempFile =>
    //         PrettyPrinting.printTraces(Set(a), tempFile.toString, false)
    //         val strOf = tempFile.contentAsString
    //         if(true || strOf.contains("setNegativeButton")){
    //             traces.addOne(strOf)
    //         }
    //     }
    // }
}
val allF = File("tmpall.txt")
traces.foreach(allF.append)

post-SerializedIRLineLoc(410,line: 410 return,1214017279)
       state: ( locals:          heap: p-1.listener->NULL * StaticPtEdge(org.connectbot.util.PubkeyDatabase,sInstance)->NULL * p-5.mEntropy->p-4          pure: p-5  !=  NULL && p-2  !=  NULL && p-4  !=  NULL && p-3  !=  NULL && p-0  !=  NULL && p-1  !=  NULL         types: List(p-0:{11230}, p-1:{6770}, p-3:{11232}, p-4:{7027,7028,7125,7126,7168 ...}, p-2:{11251}, p-5:{6618})    trace: O(CBEnter I_CBEnter_ViewonTouchEvent ( _T_,p-5 );p-0;O(CIExit I_CIExit_Dialogshow ( _T_,_T_ );p-3;O(CBEnter I_CBEnter_Runnablerun ( _T_,p-3 );p-2;O(CBEnter I_CBEnter_Runnablerun ( _T_,p-2 );O(CIEnter I_CIEnter_Dialogdismiss ( _T_,p-0 ))
post-SerializedIRLineLoc(378,line: 378 r15 := @caughtexception,1008700033)
       state: ( locals: @this -> p-1      locals:          heap: p-1.progress->p-0 * p-1.listener->NULL          pure: p-2  !=  NULL && p-1  !=  NULL         types: List(p-0:{11230}, p-1:{6770}, p-2:{11251})    trace: p-2;O(CBEnter I_CBEnter_

import scala.collection.mutable

import scala.util.Random

traces: mutable.HashSet[String] = HashSet()
allF: File = /home/notebooks/tmpall.txt

In [23]:
// Statistics of live nodes
object LiveNodes{ // Create object to supress output TODO: upgrade ammonite at some point
    val live = db.getAllLiveNodes()
    val (xLive,yLive) = live.groupBy(_.ordDepth).map(a => (a._1, a._2.size)).toList.sortBy(_._1).unzip
    //val allNodes = db.getAllNodes()
    //val (xAll,yAll) = allNodes.groupBy(_.ordDepth).map(a => (a._1, a._2.size)).toList.sortBy(_._1).unzip


    val data = Seq(
        Scatter(xLive, yLive, name="live nodes"),
    //     //Scatter(xAll,yAll, name="all nodes")
    )
    val currentlyWorkingNodes = db.getNoSucc().filter{n => n.qry.searchState.toString.contains("live") && n.subsumed.isEmpty}

}

// plot(LiveNodes.data)
// LiveNodes.data.head.plot()
println(s"number of live nodes: ${LiveNodes.live.size}")
println(s"max depth: ${LiveNodes.live.map{_.depth}.max}")
println(s"currently active nodes: ${LiveNodes.currentlyWorkingNodes.size}")
// LiveNodes.data.head

number of live nodes: 26829
max depth: 955
currently active nodes: 579


defined object LiveNodes

number of live nodes: 26566
max depth: 955
currently active nodes: 549
number of live nodes: 26566
max depth: 955
currently active nodes: 540
number of live nodes: 26491
max depth: 955
currently active nodes: 542

In [ ]:
object DbgUnreach{
    val byDepth = LiveNodes.live.toList.sortBy(_.depth).reverse
}
// File.usingTemporaryFile() {tempFile =>
//   PrettyPrinting.printTraces(Set(DbgUnreach.byDepth.head), tempFile.toString, false)
//   println(tempFile.contentAsString)
// } 


In [ ]:
object AvgPureSize{
    val (x,y) = LiveNodes.live.groupBy(_.depth).map{case (depth, nodes) => 
        val pureSizes = nodes.map(n => n.qry.state.sf.pureFormula.size)
        (depth,pureSizes.sum/nodes.size.toDouble)
    }.toList.sortBy(_._1).unzip

}
Scatter(AvgPureSize.x, AvgPureSize.y).plot()

In [ ]:
LiveNodes.live.groupBy(_.depth)(160).head.qry.state.toString 

In [ ]:
LiveNodes.live.size

In [ ]:
// LiveNodes.live.filter(pn => pn.qry.getState.get.sf.traceAbstraction.rightOfArrow.size > 0)
// LiveNodes.live.head
def groupByLocation(nodes:Iterable[DBPathNode]) = {
    
}
groupByLocation(LiveNodes.live)

In [ ]:
// Count pure vars in state


Histogram(x=List(1,1,3,5,5,5)).plot()//.withXbins(new Bins(0,1,1)).plot()

In [ ]:
def getEndLive()= {
    // Note: this cant distinguish live at end from nodes with no pred locations
    val live = db.getLive()
    val filtLive:Set[IPathNode] = live.filter{a => a.qry.isLive && a.subsumed.isEmpty}.map(_.asInstanceOf[IPathNode])
    // BounderUtil.computeDepthOfWitOrLive(filtLive, QueryFinished)
    filtLive
}

def dumpFrom(liveNodes: Set[IPathNode],outF:String):Unit = {
    import edu.colorado.plv.bounder.symbolicexecutor.state.PrettyPrinting
    val pp = new PrettyPrinting()
    val inspQ: Set[IPathNode] = liveNodes
    val pptr = pp.witnessToTrace(inspQ.toList,false)
    println(s"inspection set size: ${inspQ.size}")
    pp.dumpDebugInfo(inspQ.toSet, "timeout", true, Some(outF))
}

dumpFrom(getEndLive(), "/Users/shawnmeier/Desktop/55select/org.andstatus/sample/res_1763/specout/org.andstatus/sample")

In [ ]:
// LiveNodes.live.map{_.depth}
// dumpFrom(LiveNodes.live,"/Users/shawnmeier/Desktop/AntennapodExecute_bug_workingdir")

In [ ]:
def dumpFromDepth(liveNodes: Set[DBPathNode],outF:String, sampDepth:Int):Unit = {
    import edu.colorado.plv.bounder.symbolicexecutor.state.PrettyPrinting
    val pp = new PrettyPrinting()
    val inspQ: Set[IPathNode] = liveNodes.filter(n => n.depth == sampDepth).map(_.asInstanceOf[IPathNode])
    val pptr = pp.witnessToTrace(inspQ.toList,false)
    println(s"inspection set size: ${inspQ.size}")
    pp.dumpDebugInfo(inspQ.toSet, "timeout", true, Some(outF))
}
// pp.qryString(inspQ.qry)
// Test1.pptr.take(3)

dumpFromDepth(LiveNodes.live, 
              "/Users/shawnmeier/Desktop/AntennapodExecute_fix_workingdir", 
                38)

In [ ]:
def getEXSet() = {
    val locGroup = LiveNodes.live.groupBy(n => n.qry.loc)
    val mostVisitedList = locGroup.map{a => (a._1,a._2.size)}.toList.sortBy(a => a._2).reverse
    val mostVisitedNode = mostVisitedList.head._1
    
    val pp = new PrettyPrinting()
    val nodeList = locGroup(mostVisitedNode)
    val outf = File("/Users/shawnmeier/Desktop/db_triage/states.txt")
    nodeList.foreach{n => 
        outf.appendLine().append(pp.qryString(n.qry))
        outf.appendLine().append(s"subs: ${n.subsumedID}")
    }
}
    
getEXSet()

In [ ]:
LiveNodes.live.groupBy(n => n.subsumed.isDefined).map{a => (a._1,a._2.size)}

In [ ]:
LiveNodes.live.groupBy(_.dbgTrace).map(a => (a._1.toString,a._2.size)).toList.sortBy(_._2).reverse

In [ ]:
LiveNodes.live.filter(_.dbgTrace.toString == "Some(List(I_CBEnter_Activity_onCreate, I_CIExit_Activity_findView, FreshRef(LS__10335), I_CIExit_View_setOnClickListener, I_CBEnter_Activity_onResume, I_CIExit_Activity_finish, I_CIExit_View_setOnClickListener, I_CBEnter_ViewOnClickListener_onClick))").groupBy(_.dbgHeap).map(a => (a._1.toString,a._2.size)).toList.sortBy(_._2).reverse.map(println)

In [ ]:
object OUT32{
    import edu.colorado.plv.bounder.symbolicexecutor.state.State
    val dbgNodes = LiveNodes.live.filter(_.dbgTrace.toString == "Some(List(I_CIExit_Dialog_show, I_CIEnter_Dialog_dismiss))").groupBy(_.dbgHeap).map(a => (a._1.toString,a._2.size,a._2)).toList.sortBy(_._2).reverse
    val nodeList = OUT32.dbgNodes.head._3.toList
    def pp(s:State) = {
        val o = List("locals:","heap:","types:","trace:").foldLeft(s.toString){
            case (acc,v) => acc.replace(s"$v", s"\n$v")
        }
        println(o)
    }
    val stateList = nodeList.map(n => n.qry.getState.get)
    val stackGrouped = stateList.groupBy(_.sf.callStack.toString()).toList.sortBy(_._2.size).reverse
}
OUT32.stackGrouped.map(a => (a._1, a._2.size))
//OUT32.stateList.sortBy(s => s.sf.callStack.toString()).take(10).foreach{OUT32.pp}


In [ ]:
import upickle.default.{macroRW, read, write, ReadWriter => RW}
val js = ujson.Arr(List(List(1),List(2)))
js.toString

In [ ]:
List("fooo","bar","a").sortBy(_.length).take(5)